# TITLE: 

# By 
## Name1:
## Name2:
## Name3:

In [1]:
## Libaries
import pandas as pd
import numpy as np
import plotly.express as px
import pytz # get timezone inf
import matplotlib.pyplot as plt
import datetime


In [2]:
# Files
API_GameLogs = 'gameLogs.csv'
data = pd.read_csv(API_GameLogs)

data.tail()


# read NBA dataset, General Information
game_data = pd.read_csv('Datasets/NBA Database Datasets/game.csv')
players = pd.read_csv('Datasets/NBA Database Datasets/player.csv')
team_data = pd.read_csv('Datasets/NBA Database Datasets/team.csv')



# Basics Graphs

In [3]:
import pandas as pd
import altair as alt


# Sample code to prepare the game_data_filtered if you've not already:
game_data_filtered = pd.read_csv('Datasets/NBA Database Datasets/game.csv')


# use season year and not game date, season_id is the year of the season. It also has an extra digit in the front we need to remove 
game_data_filtered['year'] = game_data_filtered['season_id'].astype(str).apply(lambda x: int(x[1:5]))

# Group by stats for home and away teams combined
tov_data = game_data_filtered.groupby('year').agg({
    'tov_home': 'mean',
    'tov_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='tov_value')

fg_pct_data = game_data_filtered.groupby('year').agg({
    'fg_pct_home': 'mean',
    'fg_pct_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='fg_pct_value')

reb_home_data = game_data_filtered.groupby('year').agg({
    'reb_home': 'mean',
    'reb_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='reb_value')

oreb_data = game_data_filtered.groupby('year').agg({
    'oreb_home': 'mean',
    'oreb_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='oreb_value')


# Create a slider for year selection
slider = alt.binding_range(min=1990, max=tov_data['year'].max(), step=1, name='Year:')
slider_selection = alt.selection_point(bind=slider, fields=['year'], name='Selector')

# Create a 'team_type' column for coloring the bars
tov_data['team_type'] = tov_data['stat'].apply(lambda x: 'Home' if 'home' in x else 'Away')
fg_pct_data['team_type'] = fg_pct_data['stat'].apply(lambda x: 'Home' if 'home' in x else 'Away')
reb_home_data['team_type'] = reb_home_data['stat'].apply(lambda x: 'Home' if 'home' in x else 'Away')
oreb_data['team_type'] = oreb_data['stat'].apply(lambda x: 'Home' if 'home' in x else 'Away')

# Create the charts with color encoding
def create_chart(data, x_title, y_title, tooltip_fields):
    return alt.Chart(data).mark_bar().encode(
        x=alt.X('stat:N', axis=alt.Axis(title=x_title)),
        y=alt.Y(f'{y_title}:Q', axis=alt.Axis(title='Value')),
        color=alt.Color('team_type:N', scale=alt.Scale(domain=['Home', 'Away'], range=['blue', 'red'])),
        tooltip=tooltip_fields
    ).properties(
        width=200
    ).add_params(
        slider_selection
    ).transform_filter(
        slider_selection
    )

# Creating each chart
tov_Chart = create_chart(tov_data, 'Turnovers', 'tov_value', ['year', 'stat', 'tov_value'])
fg_pct_Chart = create_chart(fg_pct_data, 'Field Goal Percentage', 'fg_pct_value', ['year', 'stat', 'fg_pct_value'])
reb_Chart = create_chart(reb_home_data, 'Rebounds', 'reb_value', ['year', 'stat', 'reb_value'])
oreb_Chart = create_chart(oreb_data, 'Offesive Rebounds', 'oreb_value', ['year', 'stat', 'oreb_value'])

# Combine the charts
combined_charts_stats = alt.hconcat(tov_Chart, fg_pct_Chart, reb_Chart, oreb_Chart).resolve_scale(y='independent').properties(title='Average Game Statistics per Year')

# Display the chart
combined_charts_stats


alt.HConcatChart(...)

# Model
Only the machine learning single nuetral net

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing 
from sklearn.metrics import classification_report
import plotly.express as px

data = pd.read_csv('nbaHomeWinLossModelDataset.csv').drop(['Unnamed: 0'],axis=1)
data = data.dropna()

data_new = pd.read_csv('gameLogs.csv')

# DASH
### link: http://127.0.0.1:8050/

In [5]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import altair as alt
import dash_vega_components as dvc


app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='NBA Wins Analysis', style={'textAlign':'center'}),
    dvc.Vega(
        id='graph-content',
        opt={},
        spec=combined_charts_stats.to_dict()
    )
])

@callback(
    Output('graph-content', 'spec'),
    Input('year-slider', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug=True)